In [1]:
import pandas as pd
import numpy as np
data_dir_local = "../data/"
data_dir = "/orcd/pool/003/dbertsim_shared/ukb/"

In [2]:
DIAG_ICD_MAP = {
    "ischemic_heart_disease": {"I20", "I21", "I22", "I23", "I24", "I25"},
    "stroke": {"I60", "I61", "I62", "I63", "I64", "I65", "I66", "I67", "I68", "I69"},
    "hypertensive_heart_kidney_diseases": {"I11", "I12", "I13"}, # Renamed and grouped I10-I15 (Note: I14 does not exist)
    "heart_failure": {"I50"},
    "atrial_fibrillation": {"I48"},
    "peripheral_vascular_disease": {"I70", "I71", "I72", "I73", "I74"},
    "type_2_diabetes": {"E11"},
    "type_1_diabetes": {"E10"},
    "alzheimers_disease": {"G30"},
    "other_dementia": {"F01", "F02", "F03"},
    "parkinsons": {"G20"},
    "copd": {"J44"},
    "lower_respiratory_disease": {"J40", "J41", "J42", "J43", "J47"},
    "chronic_kidney_disease": {"N18"},
    "acute_kidney_injury": {"N17"},
    "end_stage_renal_disease": {"N19"}, # Note: N18 covers CKD. ESRD is technically N18.6 in ICD-10-CM, but N19 is unspecified kidney failure.
    "liver_disease": {"K70", "K71", "K72", "K73", "K74", "K75", "K76"},
    "depression": {"F32", "F33"},
    "anxiety_disorders": {"F40", "F41"}, 
}

In [3]:
df = pd.read_csv(f"{data_dir}blood_protein_cancers_clean.csv")
drop_cols = ['Medication for cholesterol, blood pressure or diabetes'] + [col for col in df.columns if "operation" in col.lower()] + \
    [col for col in df.columns if "cancer" in col.lower()] + [col for col in df.columns if "time_to_diagnosis" in col.lower()]
df = df.drop(columns=drop_cols)

/tmp/ipykernel_40961/672335869.py:1: DtypeWarning: Columns (13,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{data_dir}blood_protein_cancers_clean.csv")


In [4]:
def rename_columns(df, field_dict):
    # drop instances - take eid only 
    for c in set(df.columns) - {"eid"}:
        df = df.rename(columns={c: c.split("p")[1].split("_")[0]})
        
    # map from eid to name 
    for c in set(df.columns) - {"eid"}:
        df = df.rename(columns={c: field_dict[int(c)]})
        
    return df


field = pd.read_csv(f"{data_dir_local}field.tsv",sep="\t")
field_dict = dict(zip(field["field_id"], field["title"]))
category = pd.read_csv(f"{data_dir_local}category.tsv",sep="\t")
df_lab_time = rename_columns(pd.read_csv(f"{data_dir_local}time_stamps_participant.csv"),field_dict)

In [7]:

df_lab_time["Date of attending assessment centre"] = pd.to_datetime(df_lab_time["Date of attending assessment centre"], errors="coerce")
df_lab_time.sort_values(by="Date of attending assessment centre")

,eid,Basophill count acquisition time,Basophill percentage acquisition time,Date of attending assessment centre,Microalbumin in urine acquisition time,Potassium in urine acquisition time,Creatinine (enzymatic) in urine acquisition time,Sodium in urine acquisition time,Eosinophill count acquisition time,Eosinophill percentage acquisition time,...,Nucleated red blood cell count acquisition time,Nucleated red blood cell percentage acquisition time,Platelet count acquisition time,Platelet crit acquisition time,Platelet distribution width acquisition time,Red blood cell (erythrocyte) count acquisition time,Red blood cell (erythrocyte) distribution width acquisition time,Reticulocyte count acquisition time,Reticulocyte percentage acquisition time,White blood cell (leukocyte) count acquisition time
291118,5597613,NaN,NaN,2006-03-13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344878,2236411,NaN,NaN,2006-03-13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303703,4412620,NaN,NaN,2006-03-13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263118,5150560,NaN,NaN,2006-03-13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41003,3377470,NaN,NaN,2006-03-13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25520,5050490,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,2010-10-01,2015-07-17,2015-07-17,2015-07-17,2015-07-17,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,...,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z,2010-10-02T10:07:31.000Z
386284,1444903,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,2010-10-01,2015-05-06,2015-05-06,2015-05-08,2015-05-06,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,...,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z,2010-10-02T10:00:32.000Z
2629,1148965,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,2010-10-01,2015-07-17,2015-07-17,2015-07-17,2015-07-17,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,...,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z,2010-10-02T09:59:09.000Z
179567,5030700,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,2010-10-01,2015-05-06,2015-05-06,2015-05-08,2015-05-06,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,...,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z,2010-10-02T10:03:20.000Z


# Preprocess ICD10 codes

In [5]:
df_diag = pd.read_parquet(f"{data_dir_local}disease_first_occurrences.parquet")
df_diag.head()

,eid,F03_date,F02_date,E11_date,F01_date,E10_date,F33_date,F41_date,F40_date,F32_date,...,K71_date,K74_date,K76_date,K70_date,K75_date,K72_date,K73_date,N19_date,N17_date,N18_date
0,1000083,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1000380,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1001803,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1002917,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1003287,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [6]:
df_diag['eid'] = df_diag['eid'].astype(int)
df_diag = df_diag.fillna(value=np.nan)
df_diag = df.merge(df_diag,
    on="eid", how="left"
)
df_diag.head()

,eid,Age at recruitment,Sex_male,Ethnic background,Body mass index (BMI),"Systolic blood pressure, automated reading","Diastolic blood pressure, automated reading",Townsend deprivation index at recruitment,Smoking status,Alcohol intake frequency.,...,K71_date,K74_date,K76_date,K70_date,K75_date,K72_date,K73_date,N19_date,N17_date,N18_date
0,1000083,49,0,British,24.7295,116.0,71.0,-3.96,Previous,Three or four times a week,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000380,62,0,British,31.2026,124.0,81.0,-5.00,Never,Daily or almost daily,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001803,47,0,Any other white background,24.2187,98.0,57.0,2.00,Never,Never,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1002917,52,1,British,20.1477,132.0,67.0,-4.23,Current,Special occasions only,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003287,69,0,British,28.1479,166.0,61.0,6.38,Previous,Three or four times a week,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Use assessment_date from df_diag (from merge with df); fallback to df_lab_time if missing
if "assessment_date" not in df_diag.columns:
    df_diag = df_diag.merge(
        df_lab_time[["eid", "Date of attending assessment centre"]], on="eid", how="left"
    )
    df_diag = df_diag.rename(columns={"Date of attending assessment centre": "assessment_date"})
df_diag["assessment_date"] = pd.to_datetime(df_diag["assessment_date"], errors="coerce")

first_dx_list = []
for diag, icds in DIAG_ICD_MAP.items():
    date_cols = [f"{icd}_date" for icd in icds]
    dt = df_diag[date_cols].apply(lambda s: pd.to_datetime(s, errors="coerce"))
    diagnosis_dates = dt.min(axis=1)
    mask = diagnosis_dates.notna()
    if not mask.any():
        print(f"{diag} all NaN")
        continue
    first_dx_list.append(
        pd.DataFrame(
            {
                "eid": df_diag.loc[mask, "eid"].values,
                "diagnosis": diag,
                "diagnosis_date": diagnosis_dates[mask].values,
                "assessment_date": df_diag.loc[mask, "assessment_date"].values,
            }
        )
    )
first_dx = pd.concat(first_dx_list, ignore_index=True)
first_dx["diagnosis_date"] = pd.to_datetime(first_dx["diagnosis_date"], errors="coerce")
first_dx["assessment_date"] = pd.to_datetime(first_dx["assessment_date"], errors="coerce")

first_dx["time_to_diagnosis"] = (
    (first_dx["diagnosis_date"] - first_dx["assessment_date"]).dt.days / 365.25
)
buffer_years = 30 / 365.25
first_dx["baseline_present"] = (first_dx["time_to_diagnosis"] <= buffer_years).astype(int)

# Wide format
df_out2 = first_dx.pivot(
    index="eid",
    columns="diagnosis",
    values=["baseline_present", "time_to_diagnosis", "diagnosis_date"]
)

df_out2.columns = [
    f"{diag}{'' if metric == 'baseline_present' else '_' + metric}"
    for metric, diag in df_out2.columns
]
df_out2 = df_out2.reset_index()
    
df_out2 = df_out2.drop(columns = [col for col in df_out2.columns if "diagnosis_date" in col])

In [8]:
for diag in DIAG_ICD_MAP.keys():
    num = df_out2[f"{diag}_time_to_diagnosis"].notna().sum()
    print(f"{diag}: {num} occurrences")

ischemic_heart_disease: 7805 occurrences
stroke: 3774 occurrences
hypertensive_heart_kidney_diseases: 625 occurrences
heart_failure: 2888 occurrences
atrial_fibrillation: 5039 occurrences
peripheral_vascular_disease: 2959 occurrences
type_2_diabetes: 5306 occurrences
type_1_diabetes: 666 occurrences
alzheimers_disease: 773 occurrences
other_dementia: 1194 occurrences
parkinsons: 973 occurrences
copd: 3426 occurrences
lower_respiratory_disease: 2720 occurrences
chronic_kidney_disease: 4101 occurrences
acute_kidney_injury: 3395 occurrences
end_stage_renal_disease: 611 occurrences
liver_disease: 2464 occurrences
depression: 7201 occurrences
anxiety_disorders: 5113 occurrences


In [9]:
# Preprocessed df for DIAG_ICD_MAP: disease=0 if diagnosis <30 days after assessment, else disease=1; plus time to diagnosis in years
DAYS_BUFFER = 30
buffer_years = DAYS_BUFFER / 365.25

# disease = 1 if first diagnosis is within 30 days after assessment (or no diagnosis), else 0
first_dx["disease"] = (first_dx["time_to_diagnosis"] <= buffer_years).astype(int)
first_dx["disease_time_to_diagnosis"] = first_dx["time_to_diagnosis"]

# Wide format: one row per eid, columns <diag> and <diag>_time_to_diagnosis for each in DIAG_ICD_MAP
df_diag_wide = first_dx.pivot(
    index="eid",
    columns="diagnosis",
    values=["disease", "disease_time_to_diagnosis"],
)
df_diag_wide.columns = [
    f"{diag}" if metric == "disease" else f"{diag}_time_to_diagnosis"
    for metric, diag in df_diag_wide.columns
]
df_diag_wide = df_diag_wide.reset_index()

df_diag_preprocessed = df.merge(df_diag_wide, on="eid", how="left")

# Fill disease columns (no diagnosis or diagnosis <30d after assessment) with 0
diag_cols = list(DIAG_ICD_MAP.keys())
for diag in diag_cols:
    if diag in df_diag_preprocessed.columns:
        df_diag_preprocessed[diag] = df_diag_preprocessed[diag].fillna(0).astype(int)
# time_to_diagnosis columns stay as float (NaN where no diagnosis or disease=0)

df_diag_preprocessed[diag_cols].head()

,ischemic_heart_disease,stroke,hypertensive_heart_kidney_diseases,heart_failure,atrial_fibrillation,peripheral_vascular_disease,type_2_diabetes,type_1_diabetes,alzheimers_disease,other_dementia,parkinsons,copd,lower_respiratory_disease,chronic_kidney_disease,acute_kidney_injury,end_stage_renal_disease,liver_disease,depression,anxiety_disorders
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
for diag in DIAG_ICD_MAP:
    print(f"{diag}: {df_diag_preprocessed[diag].sum()}")

ischemic_heart_disease: 3298
stroke: 1070
hypertensive_heart_kidney_diseases: 193
heart_failure: 495
atrial_fibrillation: 1170
peripheral_vascular_disease: 991
type_2_diabetes: 1643
type_1_diabetes: 338
alzheimers_disease: 9
other_dementia: 45
parkinsons: 158
copd: 1228
lower_respiratory_disease: 1255
chronic_kidney_disease: 902
acute_kidney_injury: 130
end_stage_renal_disease: 300
liver_disease: 546
depression: 4750
anxiety_disorders: 2220


In [12]:
df_diag_preprocessed.to_csv(f"{data_dir}blood_protein_diseases_clean.csv", index=False)

## Train/val/test split of dataset
1. Drop protein columns that have over 30% missing
2. Split into train, validation, and test using iterative-stratification on cancer

In [13]:
# df = pd.read_csv(f"{data_dir}blood_protein_diseases_clean.csv")
df = df_diag_preprocessed

In [14]:
olink_cols = [col for col in df.columns if col.startswith("olink")]
threshold = 0.6

# Compute the fraction of missing olink values per row
row_missing_fraction = df[olink_cols].isna().mean(axis=1)

# Filter out rows with >30% missing olink values
rows_to_drop = df[row_missing_fraction >= threshold].index

preprocessed_df = df[row_missing_fraction < threshold]

# Compute the fraction of missing values for these columns
missing_fraction = preprocessed_df[olink_cols].isna().mean()

# Drop the columns with >50% missing
cols_to_drop = missing_fraction[missing_fraction >= threshold].index # only 3 columns
preprocessed_df = preprocessed_df.drop(columns=[col for col in olink_cols if col in cols_to_drop])

In [15]:
print(f"dropping patients: {len(rows_to_drop)}")
print(f"dropping cols: {len(cols_to_drop)}")

dropping patients: 398
dropping cols: 3


In [16]:
DIAG_COLS = DIAG_ICD_MAP.keys()

def bin_ttd(x):
    if pd.isna(x):         return "NA"
    if x <= 30/365.25:     return "<0"
    if 30/365.25 < x <= 1: return "0-1"   # (0,1]
    if 1 < x <= 5: return "1-5"   # (0,1]
    return ">5"

def proportions(frame, label):
    return (frame[f"{label}_strata"].value_counts(normalize=False)
            .reindex(["<0","0-1","1-5",">5","NA"])
            .fillna(0))

In [20]:
for diag in DIAG_ICD_MAP:
    total = preprocessed_df[f"{diag}_time_to_diagnosis"].notna().sum()
    print(f"{diag}: {total}")

ischemic_heart_disease: 7734
stroke: 3747
hypertensive_heart_kidney_diseases: 604
heart_failure: 2857
atrial_fibrillation: 5006
peripheral_vascular_disease: 2931
type_2_diabetes: 5265
type_1_diabetes: 656
alzheimers_disease: 770
other_dementia: 1189
parkinsons: 964
copd: 3400
lower_respiratory_disease: 2695
chronic_kidney_disease: 4055
acute_kidney_injury: 3359
end_stage_renal_disease: 590
liver_disease: 2436
depression: 7145
anxiety_disorders: 5076


In [17]:
for diag in DIAG_COLS:
    preprocessed_df[f"{diag}_strata"] = preprocessed_df[f"{diag}_time_to_diagnosis"].apply(bin_ttd)
    print(f"\n{diag}:\n", proportions(preprocessed_df, diag))


ischemic_heart_disease:
 <0      3264
0-1      203
1-5     1168
>5      3099
NA     44863
Name: ischemic_heart_disease_strata, dtype: int64

stroke:
 <0      1065
0-1       84
1-5      514
>5      2084
NA     48850
Name: stroke_strata, dtype: int64

hypertensive_heart_kidney_diseases:
 <0       183
0-1       75
1-5      196
>5       150
NA     51993
Name: hypertensive_heart_kidney_diseases_strata, dtype: int64

heart_failure:
 <0       488
0-1       74
1-5      437
>5      1858
NA     49740
Name: heart_failure_strata, dtype: int64

atrial_fibrillation:
 <0      1163
0-1      148
1-5      909
>5      2786
NA     47591
Name: atrial_fibrillation_strata, dtype: int64

peripheral_vascular_disease:
 <0       980
0-1       90
1-5      440
>5      1421
NA     49666
Name: peripheral_vascular_disease_strata, dtype: int64

type_2_diabetes:
 <0      1622
0-1      192
1-5     1027
>5      2424
NA     47332
Name: type_2_diabetes_strata, dtype: int64

type_1_diabetes:
 <0       334
0-1       22
1-5 

In [21]:
## Create train, validation, and test datasets for predicting current cancer
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
# Sanity check: compare prevalences per label in full vs train vs test
def prevalence(table, cols):
    return pd.DataFrame({
        "prevalence": [table[c].mean() for c in cols],
        "n": [table[c].sum() for c in cols]
    }, index=cols)

def multilabel_stratified_split(
    df: pd.DataFrame,
    test_size=0.4,
    random_state=42,
    time=0
):
    """
    Splits df into train/test so that each label in label_cols
    has (approximately) the same prevalence in both splits,
    accounting for multi-label rows.
    """
    # 1) Build the multi-label target matrix (n_samples x n_labels)
    # Y = df[label_cols].astype(int).to_numpy()
    df_copy = df.copy()
    Y_cols = []
    for label in DIAG_COLS:
        for strata in ["<0","0-1","1-5",">5"]:
            df_copy[f"{label}_time_to_diagnosis_{strata}"] = (df[f"{label}_strata"] == strata)
            Y_cols.append(f"{label}_time_to_diagnosis_{strata}")
    Y = df_copy[Y_cols].to_numpy()

    # 2) Set up the multi-label stratified splitter
    msss = MultilabelStratifiedShuffleSplit(
        n_splits=1, test_size=test_size, random_state=random_state
    )

    # 3) Run the split; indices refer to rows of df
    (train_idx, test_idx), = msss.split(df, Y)
    
    train_df = df.iloc[train_idx].copy()
    test_df  = df.iloc[test_idx].copy()

    train_df_copy = df_copy.iloc[train_idx].copy()
    test_df_copy  = df_copy.iloc[test_idx].copy()
    
    summary = pd.concat(
        {
            "full": prevalence(df_copy, Y_cols),
            "train": prevalence(train_df_copy, Y_cols),
            "test": prevalence(test_df_copy, Y_cols),
        },
        axis=1,
    )
    print(summary)

    return train_df, test_df

# Make the split
# train_df, validtest_df = multilabel_stratified_split(preprocessed_df, test_size=0.3)
# valid_df, test_df = multilabel_stratified_split(validtest_df, test_size=0.5)
train_df, test_df = multilabel_stratified_split(preprocessed_df, test_size=0.2)

train_df.to_csv(f"{data_dir}ukb_disease_train.csv", index=False)
# valid_df.to_csv(f"{data_dir}ukb_diag_valid.csv", index=False)
test_df.to_csv(f"{data_dir}ukb_disease_test.csv", index=False)

                                                   full            train  \
                                             prevalence     n prevalence   
ischemic_heart_disease_time_to_diagnosis_<0    0.062057  3264   0.062053   
ischemic_heart_disease_time_to_diagnosis_0-1   0.003860   203   0.003850   
ischemic_heart_disease_time_to_diagnosis_1-5   0.022207  1168   0.022197   
ischemic_heart_disease_time_to_diagnosis_>5    0.058920  3099   0.058916   
stroke_time_to_diagnosis_<0                    0.020248  1065   0.020249   
...                                                 ...   ...        ...   
depression_time_to_diagnosis_>5                0.031295  1646   0.031300   
anxiety_disorders_time_to_diagnosis_<0         0.041980  2208   0.041971   
anxiety_disorders_time_to_diagnosis_0-1        0.001806    95   0.001806   
anxiety_disorders_time_to_diagnosis_1-5        0.013233   696   0.013238   
anxiety_disorders_time_to_diagnosis_>5         0.039489  2077   0.039499   

           